In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime
print('Versão do TensorFlow: ', tf.__version__)


Versão do TensorFlow:  2.12.0


In [4]:
tf.config.get_visible_devices()
 

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
batch_size = 16
img_height = 224
img_width  = 224
data_dir  = '../dataset/'

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
   data_dir,
   validation_split=0.2,
   subset="training",
   seed=123,
   image_size=(img_height, img_width),
   batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 4620 files belonging to 2 classes.
Using 3696 files for training.
Found 4620 files belonging to 2 classes.
Using 924 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['Cars', 'People']


In [8]:
num_classes = len(class_names)
data_augmentation = tf.keras.Sequential(
[
  tf.keras.layers.RandomFlip("horizontal",
                    input_shape=(img_height,img_width,3)),
  tf.keras.layers.RandomRotation(0.1),
  tf.keras.layers.RandomZoom(0.1),
])
model = tf.keras.Sequential([
   data_augmentation,
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
  ])

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
%load_ext tensorboard
log_dir = "./logs/fit/"
csv_log_dir = log_dir+'/trainning.log'
backup = './logs/fit/backup'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

epochs=200
history = model.fit(
          train_ds,
          validation_data=val_ds,
          epochs=epochs,
          validation_steps=val_ds.cardinality().numpy(),
          callbacks=[
                tensorboard_callback,
                tf.keras.callbacks.CSVLogger(csv_log_dir),
                tf.keras.callbacks.BackupAndRestore(
                backup_dir= backup,
                save_freq  = "epoch",
                delete_checkpoint = True,
                save_before_preemption  = False
           )]
)

Epoch 5/200
231/231 [==============================] - 10s 38ms/step - loss: 0.0151 - accuracy: 0.9949 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 6/200
231/231 [==============================] - 9s 37ms/step - loss: 0.0234 - accuracy: 0.9932 - val_loss: 0.0086 - val_accuracy: 0.9968
Epoch 7/200
231/231 [==============================] - 9s 37ms/step - loss: 0.0083 - accuracy: 0.9962 - val_loss: 0.0095 - val_accuracy: 0.9968
Epoch 8/200
231/231 [==============================] - 8s 36ms/step - loss: 0.0191 - accuracy: 0.9940 - val_loss: 0.0050 - val_accuracy: 0.9978
Epoch 9/200
231/231 [==============================] - 9s 37ms/step - loss: 0.0134 - accuracy: 0.9943 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 10/200
231/231 [==============================] - 8s 37ms/step - loss: 0.0165 - accuracy: 0.9951 - val_loss: 0.0060 - val_accuracy: 0.9978
Epoch 11/200
231/231 [==============================] - 9s 37ms/step - loss: 0.0061 - accuracy: 0.9981 - val_loss: 0.0042 - val_accura

In [14]:
model.save('../model/tensorflow/car_or_people')

INFO:tensorflow:Assets written to: ../model/tensorflow/car_or_people/assets


INFO:tensorflow:Assets written to: ../model/tensorflow/car_or_people/assets
